In [5]:
import numpy as np

a = np.zeros((2,2,2))
a.ndim

3

In [1]:
import torch
print(torch.cuda.is_available() , torch.__version__)

False 2.2.0+cpu


In [2]:
# %% test a specific model
from src.interface import ModelTrainer
ModelTrainer.main(stage = 0 , resume = 0 , checkname= 1)

24-05-01 06:32:20|MOD:display     |: Model Specifics:
24-05-01 06:32:20|MOD:display     |: Start Process [Data] at Wed May  1 06:32:20 2024!


--Process Queue : Data + Fit + Test
--Start Training New!
--Model_name is set to gru_day_ShortTest.12!
Callback : DynamicDataLink() , assign / unlink dynamic data in tra networks
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer() , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=20,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100, 1]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=10) , display batch progress bar
Callback : StatusDisplay(verbosity=10) , display epoch / event information
{'random_seed': None,
 'model_name': 'gru_day_ShortTest.12',
 

24-05-01 06:32:21|MOD:display     |: Finish Process [Data], Cost 1.7 Secs
24-05-01 06:32:21|MOD:display     |: Start Process [Fit] at Wed May  1 06:32:21 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


Valid Ep#  0 score : 0.12133: 100%|██████████| 10/10 [00:02<00:00,  4.56it/s]
FirstBite Ep#  0 : loss  0.98505, train 0.01809, valid 0.12133, best 0.1213, lr1.3e-03
Valid Ep#  1 score : 0.07842: 100%|██████████| 10/10 [00:02<00:00,  4.66it/s]
FirstBite Ep#  1 : loss  0.95501, train 0.04412, valid 0.07842, best 0.1213, lr2.5e-03
Valid Ep#  2 score : 0.11272: 100%|██████████| 10/10 [00:02<00:00,  4.42it/s]
FirstBite Ep#  2 : loss  0.92590, train 0.08410, valid 0.11272, best 0.1213, lr3.8e-03
Valid Ep#  3 score : 0.09889: 100%|██████████| 10/10 [00:02<00:00,  4.34it/s]
FirstBite Ep#  3 : loss  0.90334, train 0.11483, valid 0.09889, best 0.1213, lr5.0e-03
24-05-01 06:34:22|MOD:display     |: gru_day_ShortTest.12 #0 @20170103|FirstBite Ep#  4 Max Epoch|Train 0.1148 Valid 0.0989 BestVal 0.1213|Cost  2.0Min, 24.0Sec/Ep
24-05-01 06:34:22|MOD:display     |: Finish Process [Fit], Cost 0.0 Hours, 2.0 Min/model, 30.3 Sec/Epoch
24-05-01 06:34:22|MOD:display     |: Start Process [Test] at Wed May  1

                    hook_name  num_calls  total_time   avg_time
10             on_train_batch         40   91.203262   2.280082
26              on_test_batch        105   27.487349   0.261784
20           on_fit_model_end          1   20.020389  20.020389
15        on_validation_batch         40    8.827659   0.220691
4          on_fit_model_start          1    0.886645   0.886645
23        on_test_model_start          1    0.583928   0.583928
27          on_test_batch_end        105    0.095590   0.000910
11         on_train_batch_end         40    0.044368   0.001109
16    on_validation_batch_end         40    0.040258   0.001006
24   on_test_model_type_start          3    0.021821   0.007274
30                on_test_end          1    0.020313   0.020313
17    on_validation_epoch_end          4    0.017091   0.004273
6        on_train_epoch_start          4    0.016066   0.004016
18           on_fit_epoch_end          4    0.014108   0.003527
31         on_summarize_model          1

In [ ]:
# prepare data
from src.interface import DataModule
DataModule.prepare_data()

Use device name: NVIDIA GeForce RTX 4090


24-03-31 09:52:11|MOD:data_preprocessing|: if_train is False , Data Processing start!
24-03-31 09:52:11|MOD:data_preprocessing|: 3 datas :['y', 'trade_day', 'trade_30m']


Sun Mar 31 09:52:11 2024 : y start ...
labels blocks reading ret10_lag DataBase's ...... cost 0.39 secs
labels blocks reading ret20_lag DataBase's ...... cost 0.33 secs
labels blocks merging ...... cost 0.05 secs
models blocks reading risk_exp DataBase's ...... cost 1.66 secs
models blocks merging ...... cost 0.17 secs
y blocks process ...... cost 0.88 secs
y blocks masking ...... cost 0.05 secs
y blocks saving  ...... cost 0.14 secs
y blocks norming ...... cost 0.00 secs
Sun Mar 31 09:52:14 2024 : y finished! Cost 3.67 Seconds
Sun Mar 31 09:52:14 2024 : trade_day start ...
trade blocks reading day DataBase's ...... cost 0.56 secs
trade_day blocks merging ...... cost 0.00 secs
trade_day blocks process ...... cost 0.03 secs
trade_day blocks masking ...... cost 0.05 secs
trade_day blocks saving  ...... cost 0.16 secs
trade_day blocks norming ...... cost 0.00 secs
Sun Mar 31 09:52:15 2024 : trade_day finished! Cost 0.81 Seconds
Sun Mar 31 09:52:15 2024 : trade_30m start ...
trade blocks r

24-03-31 09:52:21|MOD:data_preprocessing|: Data Processing Finished! Cost 10.02 Seconds


... cost 1.13 secs
trade_30m blocks norming ...... cost 0.00 secs
Sun Mar 31 09:52:21 2024 : trade_30m finished! Cost 5.19 Seconds
